In [1]:
import psycopg2
import pandas as pd 

In [2]:
def ConnectDatabase( database, user, password,ip = "localhost", port = 5432):
    conn = psycopg2.connect(host=ip ,database=database, user=user, password=str(password),port = port)
    conn.autocommit = True
    print("Connect Success")
    cursor = conn.cursor()
    return cursor

In [3]:
def Drop(database,table):
    database.execute("DROP table if exists {} CASCADE;".format(table))

In [4]:
def CreateUniversity(database):
    print("Create Table university.")
          
    query = '''Create Table university(
                    id CHAR(3) PRIMARY KEY, 
                    name VARCHAR NOT NULL,
                    position VARCHAR NOT NULL 
                    );'''

    database.execute(query)
    

In [5]:
def InsertUniversity(database):
    print("Insert Data into university.\n")
    
    nameID = pd.read_csv("../datas/name2ID.csv")
    namePos = pd.read_csv("../datas/alluniversity.csv")
    #merge two data
    data = list()
    for (i,j) in zip(list(nameID['name']),list(nameID['ID'])):
        for (name,pos)   in zip(list(namePos['學校名稱']),list(namePos['學校地址'])):
            if(i in name):
                data.append((j,i,pos))
    #create query 
    query = '''INSERT INTO university \n Values '''
   
    for (ID,name,pos) in data:
        query+= "('"+str(ID).zfill(3) +"','"+name +"','"+pos+"')"+",\n"
    query = query[:-2]
    query += ';'
   
    database.execute(query)

In [6]:
def CreateAcademy(database):
    print("Create Table academy.")
    
    query = '''Create Table academy(
                    name VARCHAR PRIMARY KEY 
                   
                    );'''

    database.execute(query)
    
    

In [7]:
def InsertAcademy(database):
    print("Insert Data into academy.\n")
    
    aca = pd.read_csv("../datas/Group.csv")
    query = '''INSERT INTO academy \n Values '''
    
    for i in aca['Group']:
        query+= "('"+i+"'),\n"
    query = query[:-2]
    
    
    database.execute(query)

In [8]:
def CreateMainDepartment(database):
    print("Create Table mainDepartment.")
    
    query = '''Create Table mainDepartment(
                    academy VARCHAR NOT NULL,
                    department VARCHAR NOT NULL,
                    subject VARCHAR NOT NULL,
                    
                    FOREIGN KEY (academy) REFERENCES academy(name),
                    PRIMARY KEY (academy, department) 
                   
                    );'''

    database.execute(query)
    

In [9]:
def InsertMainDepartment(database):
    print("Insert Data into mainDepartment.\n")
    
    md =  pd.read_csv("../datas/Group_Department_Subject.csv")
    query = '''INSERT INTO MainDepartment \n Values '''
    
    for (i,j,k) in zip(md['Group'],md['Department'],md['Subjects']):
        query+= "('"+i+"','"+j+"','"+k+"'),\n"
    query = query[:-2]
    
    
    database.execute(query)

In [10]:
def CreateDepartment(database):
    print("Create Table department.")
 
    query = '''Create Table department(
                    id VARCHAR NOT NULL,
                    name VARCHAR NOT NULL,
                    
                    university_id CHAR(3) NOT NULL,
                    academy VARCHAR NOT NULL,
                    
                    PRIMARY KEY (id), 
                    FOREIGN KEY (academy) REFERENCES academy(name),
                    FOREIGN KEY (university_id) REFERENCES university(id)
                   
                    );'''    
    database.execute(query)

In [79]:
def  InsertDepartment(database):
    print("Insert Data into department.\n")
    
    df1 = pd.read_csv('../datas/學測output.csv')
    df2 = pd.read_csv('../datas/指考.csv')
    df3 = pd.read_csv("../datas/department2group.csv")
    df4 = pd.read_csv("../datas/name2ID.csv")
    
    name2ID = dict()
    
    for i in zip(df4['ID'],df4['name']):
        name2ID[i[1]] = str(i[0]).zfill(3)
  
    department2group = dict()
    
    for j in df3['University']:
        department2group[name2ID[j]] = dict()
    
    for (i,j,k) in zip( df3['University'],df3['DEPARTMENT'],df3['(code)Group']):
         department2group[name2ID[i]][j] = k.split(')')[1][1:-2]
        

    query = '''INSERT INTO department \n Values '''
    
    for i in dep:
        for j in dep[i]: 
            if(department2group.get(i,None)!=None):
                for k in department2group[i]:
                    if(j in k): 
                        query+= "('"+i+str(dep[i][j]).zfill(3) +"','"+j +"','"+i+"',"+department2group[i][k]+"')"+",\n"
                        break
    query = query[:-2]
    query += ';'
    print(query)

    database.execute(query)

In [67]:
# df1 = pd.read_csv("../datas/Group_Department.csv")

In [70]:
InsertDepartment(db)

NameError: name 'db' is not defined

In [38]:
'電影與創意媒體學系 ' in '電影與創意媒體學系(電影組)'

False

In [37]:
set(list(df1['Group']))

{'地球與環境',
 '外語',
 '大眾傳播',
 '工程',
 '建築與設計',
 '教育',
 '數理化',
 '文史哲',
 '法政',
 '生命科學',
 '生物資源',
 '社會與心理',
 '管理',
 '藝術',
 '財經',
 '資訊',
 '遊憩與運動',
 '醫藥衛生'}

In [22]:
df = pd.read_csv("../datas/department2group.csv")

In [15]:
df

NameError: name 'df' is not defined

In [14]:
df1 = pd.read_csv('../datas/學測output.csv')
df2 = pd.read_csv('../datas/指考.csv')
    

dep  = dict() 

for i in df2['代碼']:
    dep[str(i).zfill(5)[:3]]=  dict()
for i in df1['Code']:
    dep[str(i).zfill(6)[:3]]=  dict()

for (i,j) in zip(df2['代碼'],df2['學系']):
    dep[str(i).zfill(5)[:3]][j] = 0
for (i,j) in zip(df1['Code'],df1['name']):
    dep[str(i).zfill(6)[:3]][j] =  0

for i in dep:
    for (k,l) in zip(range(1,len(dep[i])+1),dep[i].keys()):
        dep[i][l]=k

In [ ]:
dep['001']

In [ ]:
tt.sort()

In [44]:
table = dict() 
for i in zip(range(6),['--','底','後','均','前','頂']):
    table[i[1]] = i[0]

In [45]:
table

{'--': 0, '底': 1, '後': 2, '均': 3, '前': 4, '頂': 5}

In [13]:
def CreateGSAT(database):
    print("Create Table GSAT.")
 
    query = '''Create Table GSAT(
                    id VARCHAR NOT NULL,
                    quta SMALLINT,
                    
                    chinese SMALLINT NOT NULL,
                    english SMALLINT  NOT NULL,
                    math    SMALLINT NOT NULL,
                    society SMALLINT  NOT NULL,
                    science SMALLINT NOT NULL,
                    
                    
                    FOREIGN KEY (id) REFERENCES department(id),
                    PRIMARY KEY (id) 
                   
                    );'''

    database.execute(query)
    

In [76]:
def  InsertGSAT(database):
    print("Insert Data into GSAT.\n")
    
    table = dict() 
    for i in zip(range(6),['--','底','後','均','前','頂']):
        table[i[1]] = str(i[0])
    
    df1 = pd.read_csv('../datas/學測output.csv')
    
    

   
            
    query = '''INSERT INTO GSAT \n Values '''
    
    for i in zip(df1['Code'],df1['name'],df1['num'],df1['s_Chinese'],df1['s_English'],df1['s_Math'],df1['s_Society'],df1['s_Science']):               
        code = str(i[0]).zfill(6)[:3]
      
        temp = list()
        for index in range(3,8):
            temp.append(table[i[index]])
        query+= "('"+code+str(dep[code][i[1]]).zfill(3) +"',"+ str(i[2])+","+ ','.join(temp)+")"+",\n"
        
    query = query[:-2]
    query += ';'
   
    database.execute(query)

In [39]:
df = pd.read_csv('../datas/學測output.csv')

In [40]:
df

,Code,name,num,s_Chinese,s_English,s_Math,s_Society,s_Science,m_Chinese,m_English,m_Math,m_Society,m_Science
0,1012,中國文學系,20,前,均,均,均,--,3,6,--,8,--
1,1022,外國語文學系,46,前,頂,--,前,--,--,2.5,--,--,--
2,1032,歷史學系,24,前,前,均,前,--,3,3,--,3,--
3,1042,哲學系,20,前,前,前,--,--,6,3,6,--,--
4,1052,人類學系,13,--,均,--,均,--,--,5,--,--,--
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1983,153152,護理學系,18,--,後,--,--,後,--,4,--,--,3
1984,153162,長期照護學系,20,後,--,--,--,--,3,--,--,--,--
1985,153172,社會工作學系,20,--,--,--,均,--,--,--,--,3,--
1986,153182,都市計畫與景觀學系,20,後,--,--,--,--,5,--,3,4,--


def CreateAST(database):
    print("Create Table AST.")
    
   
    query = '''Create Table AST(
                    id VARCHAR NOT NULL,
                    quta SMALLINT,
                    
                    chinese FLOAT NOT NULL,
                    english FLOAT NOT NULL,
                    math1   FLOAT NOT NULL,
                    math2  FLOAT NOT NULL,
                    
                    physical FLOAT NOT NULL,
                    chemistry FLOAT NOT NULL,
                    biological FLOAT NOT NULL,
                    
                    history FLOAT NOT NULL,
                    geography FLOAT NOT NULL,
                    citizen FLOAT NOT NULL,
                    
                    
                    FOREIGN KEY (id) REFERENCES department(id),
                    PRIMARY KEY (id) 
                   
                    );'''

    database.execute(query)
    

def CreateCareer(database):
    print("Create Table career.")
    
      query = '''Create Table career(
                    id VARCHAR NOT NULL,
                    career1 VARCHAR ,
                    career2 VARCHAR,
                    career2 VARCHAR,
                    
                    
                    FOREIGN KEY (id) REFERENCES depatment(id),
                    PRIMARY KEY (id) 
                   
                    );'''

    database.execute(query)
    

def CreateFuture(database):
    print("Create Table future.")
    query = '''Create Table career(
                    id VARCHAR NOT NULL,
                    work_ratio TINYINT ,
           
                    FOREIGN KEY (id) REFERENCES depatment(id),
                    PRIMARY KEY (id) 
                   
                    );'''

    database.execute(query)
    

In [ ]:
import pandas as pd
df = pd.read_excel("../datas/count.xlsx")


In [ ]:
df.dropna()

In [ ]:
df = pd.read_csv("../datas/學測output.csv")

In [ ]:
df

In [ ]:
CreateMainDepartment(db)

In [ ]:
InsertMainDepartment(db)

In [17]:
Drop(db,"mainDepartment")

In [71]:
db = ConnectDatabase("schools", "postgres", 123456, "localhost",5433)

Connect Success


In [72]:
tables = ['university','academy','mainDepartment','department','GSAT']

In [77]:
for i in tables:
    Drop(db,i)

In [78]:
def run():
    database = ConnectDatabase("schools", "postgres", 123456, "localhost",5433)
   
    CreateUniversity(database)
    InsertUniversity(database)
        
    CreateAcademy(database)
    InsertAcademy(database)
    
    CreateMainDepartment(database)
    InsertMainDepartment(database)
    
    CreateDepartment(database)
    InsertDepartment(database)
    
    CreateGSAT(db)
    InsertGSAT(db)
run()

Connect Success
Create Table university.
Insert Data into university.

Create Table academy.
Insert Data into academy.

Create Table mainDepartment.
Insert Data into mainDepartment.

Create Table department.
Insert Data into department.



SyntaxError: syntax error at or near "005002"
LINE 3: ('005002','歷史學系','005',文史哲'),
          ^


In [80]:
  InsertDepartment(db)

Insert Data into department.

INSERT INTO department 
 Values ('005001','中國文學系','005',文史哲'),
('005002','歷史學系','005',文史哲'),
('005003','哲學系','005',文史哲'),
('005004','政治學系','005',法政'),
('005005','社會學系','005',社會與心理'),
('005006','社會工作學系','005',社會與心理'),
('005007','英文學系','005',外語'),
('005008','日本語文學系','005',外語'),
('005009','德國文化學系','005',外語'),
('005010','數學系','005',數理化'),
('005011','物理學系','005',數理化'),
('005012','化學系','005',外語'),
('005013','微生物學系應用微生物組','005',生命科學'),
('005014','心理學系','005',社會與心理'),
('005015','法律學系','005',法政'),
('005016','經濟學系','005',財經'),
('005017','會計學系','005',財經'),
('005018','企業管理學系','005',管理'),
('005019','國際經營與貿易學系','005',財經'),
('005020','財務工程與精算數學系','005',數理化'),
('005021','資訊管理學系','005',資訊'),
('005022','微生物學系生物科技組','005',生命科學'),
('005023','巨量資料管理學院學士學位學程','005',數理化'),
('003001','中國文學系','003',文史哲'),
('003002','外國語文學系','003',外語'),
('003003','財務金融學系','003',財經'),
('003004','企業管理學系','003',管理'),
('003005','法律學系','003',法政'),
('003006','園藝學系','003',生物資源'),
('003007','歷史學系','003',文史

SyntaxError: syntax error at or near "005002"
LINE 3: ('005002','歷史學系','005',文史哲'),
          ^


In [76]:
CreateGSAT(db)
InsertGSAT(db)

Insert Data into GSAT.

